In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf

### Default variable settings

In [2]:
PRIMARY_TICKER = 'MSFT'
PAIR_TICKER = 'TGT'

TICKERS = [PRIMARY_TICKER, PAIR_TICKER]
PERIOD = '2y'
INTERVAL = '1d'
WINDOW = 20

### Plot Strategy

In [13]:
def run_strategy(primary_ticker = PRIMARY_TICKER, 
                 pair_ticker = PAIR_TICKER, 
                 period = PERIOD, 
                 interval = INTERVAL, 
                 window = WINDOW, 
                 plot_daily_returns = True):
    '''
    Arguments (all optional)
    primary_ticker: ticker to send buy/sell signal
    pair_ticker: ticker to pair with primary_ticker
    period: time period to get data for
    window: rolling window for mean reversion strategy
    plot_daily_returns: boolean to enable/disable plotting daily returns for the tickers

    Return
    None

    Plots returns for the pair of tickers using the strategy
    '''
    data = yf.download([primary_ticker, pair_ticker], period=PERIOD, interval=INTERVAL)['Close']

    # Compute daily percentage returns
    returns = data.pct_change().dropna()

    # Compute rolling mean and standard deviation for mean reversion strategy
    zscore = (data - data.rolling(WINDOW).mean()) / data.rolling(WINDOW).std()

    # Define trading strategy
    long_signal = zscore[PRIMARY_TICKER] < -1   
    short_signal = zscore[PRIMARY_TICKER] > 1

    # Positions
    positions = pd.DataFrame(index=data.index)
    positions[PRIMARY_TICKER] = np.where(long_signal, 1, np.where(short_signal, -1, 0))
    positions[PAIR_TICKER] = -positions[PRIMARY_TICKER]  

    # Calculate strategy returns
    strategy_returns = (positions.shift(1) * returns).sum(axis=1)
    cumulative_returns = (1 + strategy_returns).cumprod()

    
    # Plot 1: Daily Percentage Returns of TGT and XLK
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=returns.index, y=returns[PRIMARY_TICKER], mode='lines', name=f'{PRIMARY_TICKER} Daily % Return'))
    fig1.add_trace(go.Scatter(x=returns.index, y=returns[PAIR_TICKER], mode='lines', name=f'{PAIR_TICKER} Daily % Return'))
    fig1.update_layout(
        title=f"Daily % Returns of {PRIMARY_TICKER} and {PAIR_TICKER} (Last 2 Years)",
        xaxis_title="Date",
        yaxis_title="Daily % Return",
        template="plotly_dark"
    )
    if plot_daily_returns: fig1.show()

    # Plot 1: Cumulative Returns of the Strategy
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name='Strategy Cumulative Returns'))
    fig2.update_layout(
        title=f"Mean Reversion Strategy: Long {PRIMARY_TICKER}, Short {PAIR_TICKER} (Last {PERIOD} Years)",
        xaxis_title="Date",
        yaxis_title="Cumulative Returns",
        template="plotly_dark"
    )
    fig2.show()

### Run function

In [16]:
run_strategy(plot_daily_returns=True)

[*********************100%***********************]  2 of 2 completed
